# Pytorch로 딥러닝 제대로 배우기-중급
## Part5-2: 과대적합 & 과소적합
## 목차

1. 데이터 호출(Cifar-10)
2. 모델
3. Weight decay (L2 penalty)

### (1) 데이터 호출 (Cifar-10)

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose

In [2]:
## Transform 지정 - ** 초급강좌에서 설명 **
transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size와 학습의 관계? 
batch_size = 64

trainset = datasets.CIFAR10(root='./dataset', train=True,
                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./dataset', train=False,
                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### (2) 모델

In [3]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(32*32*3, 1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)


### (3) 최적화(Optimization)

#### 학습/평가 함수 정의

In [4]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [5]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### SGD

#### Without penalty

In [6]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309335  [    0/50000]
loss: 2.305044  [ 6400/50000]
loss: 2.304609  [12800/50000]
loss: 2.294631  [19200/50000]
loss: 2.303111  [25600/50000]
loss: 2.299413  [32000/50000]
loss: 2.293840  [38400/50000]
loss: 2.300522  [44800/50000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.294243 

Epoch 2
-------------------------------
loss: 2.300594  [    0/50000]
loss: 2.300847  [ 6400/50000]
loss: 2.287667  [12800/50000]
loss: 2.287684  [19200/50000]
loss: 2.285975  [25600/50000]
loss: 2.285868  [32000/50000]
loss: 2.287471  [38400/50000]
loss: 2.273639  [44800/50000]
Test Error: 
 Accuracy: 18.8%, Avg loss: 2.283407 

Epoch 3
-------------------------------
loss: 2.282539  [    0/50000]
loss: 2.274345  [ 6400/50000]
loss: 2.270647  [12800/50000]
loss: 2.276626  [19200/50000]
loss: 2.281738  [25600/50000]
loss: 2.275967  [32000/50000]
loss: 2.277824  [38400/50000]
loss: 2.279803  [44800/50000]
Test Error: 
 Accuracy: 21.9%, Avg loss: 2.268418 

Epoc

#### L2 penalty 0.05

In [8]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=0.05)

In [9]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315030  [    0/50000]
loss: 2.315473  [ 6400/50000]
loss: 2.300351  [12800/50000]
loss: 2.306590  [19200/50000]
loss: 2.288002  [25600/50000]
loss: 2.295419  [32000/50000]
loss: 2.295757  [38400/50000]
loss: 2.296340  [44800/50000]
Test Error: 
 Accuracy: 17.0%, Avg loss: 2.294777 

Epoch 2
-------------------------------
loss: 2.292228  [    0/50000]
loss: 2.300972  [ 6400/50000]
loss: 2.291825  [12800/50000]
loss: 2.294820  [19200/50000]
loss: 2.291165  [25600/50000]
loss: 2.291907  [32000/50000]
loss: 2.282988  [38400/50000]
loss: 2.294231  [44800/50000]
Test Error: 
 Accuracy: 19.1%, Avg loss: 2.287262 

Epoch 3
-------------------------------
loss: 2.294825  [    0/50000]
loss: 2.286857  [ 6400/50000]
loss: 2.291963  [12800/50000]
loss: 2.283239  [19200/50000]
loss: 2.284880  [25600/50000]
loss: 2.281405  [32000/50000]
loss: 2.267808  [38400/50000]
loss: 2.277424  [44800/50000]
Test Error: 
 Accuracy: 20.3%, Avg loss: 2.280031 

Epoc

#### L2 penalty 0.1

In [10]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=0.1)

In [11]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303780  [    0/50000]
loss: 2.302180  [ 6400/50000]
loss: 2.298307  [12800/50000]
loss: 2.294475  [19200/50000]
loss: 2.298793  [25600/50000]
loss: 2.296794  [32000/50000]
loss: 2.304544  [38400/50000]
loss: 2.297290  [44800/50000]
Test Error: 
 Accuracy: 14.1%, Avg loss: 2.296496 

Epoch 2
-------------------------------
loss: 2.297639  [    0/50000]
loss: 2.295343  [ 6400/50000]
loss: 2.295147  [12800/50000]
loss: 2.294155  [19200/50000]
loss: 2.296863  [25600/50000]
loss: 2.295799  [32000/50000]
loss: 2.294973  [38400/50000]
loss: 2.292992  [44800/50000]
Test Error: 
 Accuracy: 16.6%, Avg loss: 2.293472 

Epoch 3
-------------------------------
loss: 2.289555  [    0/50000]
loss: 2.293320  [ 6400/50000]
loss: 2.291627  [12800/50000]
loss: 2.293296  [19200/50000]
loss: 2.294540  [25600/50000]
loss: 2.296237  [32000/50000]
loss: 2.293208  [38400/50000]
loss: 2.291574  [44800/50000]
Test Error: 
 Accuracy: 19.2%, Avg loss: 2.291969 

Epoc

#### L2 penalty 0.9

In [12]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=0.9)

In [13]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315534  [    0/50000]
loss: 2.301179  [ 6400/50000]
loss: 2.297627  [12800/50000]
loss: 2.302070  [19200/50000]
loss: 2.299445  [25600/50000]
loss: 2.298323  [32000/50000]
loss: 2.304456  [38400/50000]
loss: 2.304333  [44800/50000]
Test Error: 
 Accuracy: 11.2%, Avg loss: 2.302286 

Epoch 2
-------------------------------
loss: 2.302500  [    0/50000]
loss: 2.301929  [ 6400/50000]
loss: 2.303484  [12800/50000]
loss: 2.302308  [19200/50000]
loss: 2.300865  [25600/50000]
loss: 2.303806  [32000/50000]
loss: 2.302969  [38400/50000]
loss: 2.303734  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302586 

Epoch 3
-------------------------------
loss: 2.302727  [    0/50000]
loss: 2.301477  [ 6400/50000]
loss: 2.303550  [12800/50000]
loss: 2.302285  [19200/50000]
loss: 2.303658  [25600/50000]
loss: 2.302494  [32000/50000]
loss: 2.302154  [38400/50000]
loss: 2.303035  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302591 

Epoc

#### L2 penalty 10.0

In [14]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=10.0)

In [15]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302969  [    0/50000]
loss: 2.303428  [ 6400/50000]
loss: 2.301558  [12800/50000]
loss: 2.302618  [19200/50000]
loss: 2.302498  [25600/50000]
loss: 2.302521  [32000/50000]
loss: 2.302508  [38400/50000]
loss: 2.302568  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302584 

Epoch 2
-------------------------------
loss: 2.302566  [    0/50000]
loss: 2.302591  [ 6400/50000]
loss: 2.302525  [12800/50000]
loss: 2.302581  [19200/50000]
loss: 2.302601  [25600/50000]
loss: 2.302615  [32000/50000]
loss: 2.302581  [38400/50000]
loss: 2.302553  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302585 

Epoch 3
-------------------------------
loss: 2.302550  [    0/50000]
loss: 2.302593  [ 6400/50000]
loss: 2.302515  [12800/50000]
loss: 2.302557  [19200/50000]
loss: 2.302602  [25600/50000]
loss: 2.302590  [32000/50000]
loss: 2.302605  [38400/50000]
loss: 2.302626  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302586 

Epoc

#### Adam

#### Without Penalty

In [16]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [17]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303042  [    0/50000]
loss: 1.744348  [ 6400/50000]
loss: 1.780145  [12800/50000]
loss: 1.696631  [19200/50000]
loss: 1.514848  [25600/50000]
loss: 1.808943  [32000/50000]
loss: 1.635035  [38400/50000]
loss: 1.341475  [44800/50000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 1.517706 

Epoch 2
-------------------------------
loss: 1.592357  [    0/50000]
loss: 1.390438  [ 6400/50000]
loss: 1.309478  [12800/50000]
loss: 1.445026  [19200/50000]
loss: 1.506022  [25600/50000]
loss: 1.497357  [32000/50000]
loss: 1.552828  [38400/50000]
loss: 1.282064  [44800/50000]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.413745 

Epoch 3
-------------------------------
loss: 1.279614  [    0/50000]
loss: 1.005332  [ 6400/50000]
loss: 1.347215  [12800/50000]
loss: 1.282881  [19200/50000]
loss: 1.193982  [25600/50000]
loss: 1.585523  [32000/50000]
loss: 1.401633  [38400/50000]
loss: 1.315756  [44800/50000]
Test Error: 
 Accuracy: 51.0%, Avg loss: 1.400603 

Epoc

#### L2 Penalty 0.1

In [18]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)

In [19]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296900  [    0/50000]
loss: 2.301847  [ 6400/50000]
loss: 2.304766  [12800/50000]
loss: 2.302158  [19200/50000]
loss: 2.302827  [25600/50000]
loss: 2.303897  [32000/50000]
loss: 2.303024  [38400/50000]
loss: 2.302945  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302610 

Epoch 2
-------------------------------
loss: 2.302652  [    0/50000]
loss: 2.301598  [ 6400/50000]
loss: 2.303185  [12800/50000]
loss: 2.303039  [19200/50000]
loss: 2.301395  [25600/50000]
loss: 2.302690  [32000/50000]
loss: 2.302128  [38400/50000]
loss: 2.303535  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302615 

Epoch 3
-------------------------------
loss: 2.302850  [    0/50000]
loss: 2.300687  [ 6400/50000]
loss: 2.302114  [12800/50000]
loss: 2.301850  [19200/50000]
loss: 2.300807  [25600/50000]
loss: 2.302371  [32000/50000]
loss: 2.301428  [38400/50000]
loss: 2.301842  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302602 

Epoc

#### L2 Penalty 0.001

In [20]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)

In [21]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305383  [    0/50000]
loss: 1.623681  [ 6400/50000]
loss: 1.690831  [12800/50000]
loss: 1.708082  [19200/50000]
loss: 1.826439  [25600/50000]
loss: 1.782346  [32000/50000]
loss: 1.566999  [38400/50000]
loss: 1.542436  [44800/50000]
Test Error: 
 Accuracy: 43.0%, Avg loss: 1.594529 

Epoch 2
-------------------------------
loss: 1.554500  [    0/50000]
loss: 1.456655  [ 6400/50000]
loss: 1.635489  [12800/50000]
loss: 1.517817  [19200/50000]
loss: 1.627399  [25600/50000]
loss: 1.509731  [32000/50000]
loss: 1.450191  [38400/50000]
loss: 1.632089  [44800/50000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.598111 

Epoch 3
-------------------------------
loss: 1.422689  [    0/50000]
loss: 1.450999  [ 6400/50000]
loss: 1.321579  [12800/50000]
loss: 1.312236  [19200/50000]
loss: 1.554786  [25600/50000]
loss: 1.561555  [32000/50000]
loss: 1.417963  [38400/50000]
loss: 1.395072  [44800/50000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 1.486144 

Epoc